In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Plots spatial maps of reconstruction skill through comparison with ERA5
def tempPlot(): 
    means = np.load('ensemble_mean_pr_sfc_Amon.npz', allow_pickle=True)
    means2 = np.load('ensemble2_mean_pr_sfc_Amon.npz', allow_pickle=True)
    
    print("hello")
    
    newNC = nc.Dataset('adaptor.mars.internal-1663556282.2356336-8299-12-f0261241-78f1-43ef-8b90-b2d6b9b5d118.nc')
    
    subsample = newNC.variables['tp'][:, 0:121:8, 0:1440:8]
    
    maskKey = subsample[0, 0, 0]
    for i in range(16):
        for j in range(180):
            if subsample[0, i, j] == maskKey:
                subsample[:, i, j] = np.nan
                
    predictYear = 0
                
    tempsAvg = np.mean(subsample[264:504], axis=0)
    temps00 = np.mean(subsample[(predictYear*12-108):(predictYear*12-96)], axis=0)
    
    era5 = np.zeros((20, 16, 180))
    #print("hi")
    for i in range(20):
        era5[i] = np.mean(subsample[(i*12+264):(i*12+276)], axis=0) * 288000

    era5 = np.subtract(era5, tempsAvg)
    era5 = np.flip(era5, axis=1)
    era5 = np.roll(era5, 90, axis=2)
    
    
    
    coeffs = np.zeros((16, 180))
    coeffs2 = np.zeros((16, 180))
    
    ceNum = np.zeros((16, 180))
    ceDen = np.zeros((16, 180))
    ce = np.zeros((16, 180))
    print("hi")

    
    improv1 = np.zeros((16, 180))

    for i in range(16):
        for j in range(180):
            #era5[:, i, j] = np.add(era5[:, i, j], np.mean(np.multiply(means['xam'][81:101, i, j], 31536), axis=0))
            
            corr = np.corrcoef(means['xam'][24:, i, j], era5[:, i, j])[0,1]
            coeffs[i, j] = corr
            
            corr2 = np.corrcoef(means2['xam'][24:, i, j], era5[:, i, j])[0,1]
            coeffs2[i, j] = corr2
            
            
    corrChange = coeffs2 - coeffs
    
    
    for i in range(16):
        for j in range(180):
            #era5[:, i, j] = np.add(era5[:, i, j], np.mean(np.multiply(means['xam'][24:, i, j], 31536000), axis=0))
            
            #corr = np.corrcoef(means['xam'][24:, i, j], era5[:, i, j])[0,1]
            #coeffs[i, j] = corr
            
            
            ceTop = np.zeros(20)
            ceBot = np.zeros(20)
            
            for k in range(20):
                ceTop[k] = ((era5[k, i, j] - np.multiply(means['xam'][k, i, j], 31536000)) ** 2)
                #ceTop[k] = ((era5[k, i, j] - np.multiply(means['xam'][k, i, j], 12/365)) ** 2)
                ceBot[k] = ((era5[k, i, j] - np.mean(era5[:, i, j], axis=0)) ** 2)
                #print(ceTop[k])

            ceNum[i, j] = np.sum(ceTop, axis=0)
            ceDen[i, j] = np.sum(ceBot, axis=0)
            ce[i, j] = 1 - (float(ceNum[i, j]) / float(ceDen[i, j]))
            print(ce[i,j])


            if ce[i,j] < -1.0:
                ce[i, j] = -1.0
            
    
    

    """
    for j in range(10):
        areaAvgs = np.zeros(16)
        for i in range(16):
            areaAvgs[i] = np.mean(np.multiply(means['xam'][j+81, i, :], 31536), axis=0) * ((-20 * (15-i) + 300) * 0.01941 + 1)
        #print(areaAvgs[i])
        #print(np.true_divide(np.sum(areaAvgs), 55.761))
        #naivAvgs[i] = np.nanmean(absNaiv[i]) * ((-20 * i + 300) * 0.01941 + 1)
    """
    

 
    
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    #ax = plt.axes(projection=ccrs.PlateCarree())
    
    ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())
    #ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())
    ax.gridlines()
    
    plt.rcParams['figure.figsize'] = [20, 20]


    plt.contourf(means['lon'][0:16], means['lat'][0:16], ce, vmin=-1.0, vmax=1.0, levels=20, transform=ccrs.PlateCarree(), cmap=plt.cm.seismic)
    plt.colorbar(orientation="horizontal", fraction=0.05, shrink=0.9, cmap=plt.cm.seismic)

    plt.title("1960-1980")
    
    ax.coastlines()
    
    plt.show()
    
    
    
    

    
    
    
tempPlot()